In [1]:
import pandas as pd
import numpy as np
user_table = pd.read_csv('./Dataset/userlist_table.csv')
serv_table = pd.read_csv('./Dataset/wslist_table.csv')
density = 0.01

In [2]:
def multi_hot(len, idx):
    vec = [0.] * len
    for id in idx:
        vec[id] = 1.
    return vec

Uctx2Idx = {}
Idx2Uctx = {}
uidx = 0
usrc = []
udst = []

ufeats = []

for idx, item in list(user_table.iterrows()):
    UserID = item['[User ID]']
    Country = item['[Country]']
    AS = item['[AS]']

    if Country not in Uctx2Idx:
        Uctx2Idx[Country] = uidx
        Idx2Uctx[uidx] = Country
        uidx += 1

    if AS not in Uctx2Idx:
        Uctx2Idx[AS] = uidx
        Idx2Uctx[uidx] = AS
        uidx += 1

    Country = Uctx2Idx[Country]
    AS = Uctx2Idx[AS]

    usrc.append(UserID)
    udst.append(Country)

    usrc.append(UserID)
    udst.append(AS)
    ufeats.append(multi_hot(200, [Country, AS]))

Sctx2Idx = {}
Idx2Sctx = {}
sidx = 0
ssrc = []
sdst = []
sfeats = []
for row in list(serv_table.iterrows()):
    row = row[1]
    ServID = row['[Service ID]']
    Country = row['[Country]']
    AS = row['[AS]']
    Provider = row['[Service Provider]']

    if Country not in Sctx2Idx:
        Sctx2Idx[Country] = sidx
        Idx2Sctx[sidx] = Country
        sidx += 1

    if AS not in Sctx2Idx:
        Sctx2Idx[AS] = sidx
        Idx2Sctx[sidx] = AS
        sidx += 1

    if Provider not in Sctx2Idx:
        Sctx2Idx[Provider] = sidx
        Idx2Sctx[sidx] = Provider
        sidx += 1

    Country = Sctx2Idx[Country]
    AS = Sctx2Idx[AS]
    Provider = Sctx2Idx[Provider]

    ssrc.append(ServID)
    sdst.append(Country)

    ssrc.append(ServID)
    sdst.append(AS)

    ssrc.append(ServID)
    sdst.append(Provider)
    sfeats.append(multi_hot(3800, [Country, AS, Provider]))

In [3]:
from tqdm import *
rtMatrix = np.loadtxt('./Dataset/rtMatrix.txt')
tpMatrix = np.loadtxt('./Dataset/tpMatrix.txt')

# Filtering the invalid entries.
msrc, mdst = [], []

rts = []
tps = []
srcx = []
dstx = []

# 建立总图
for i in trange(339):
    for j in range(5825):
        if 0 < rtMatrix[i][j] < 19.9 and 0 < tpMatrix[i][j] < 1000:
            msrc.append(i)
            mdst.append(j)
            rts.append(rtMatrix[i][j])
            tps.append(tpMatrix[i][j])

100%|██████████| 339/339 [00:02<00:00, 133.63it/s]


# 准备建图

In [4]:
"""
    usrc: 用户索引
    udst: 地址索引
    ssrc: 服务索引
    sdst: 服务地址
    msrc：用户
    mdst：服务
"""
len(usrc), len(udst), len(msrc), len(mdst)


(678, 678, 1827436, 1827436)

In [5]:
usrc

[0,
 0,
 1,
 1,
 2,
 2,
 3,
 3,
 4,
 4,
 5,
 5,
 6,
 6,
 7,
 7,
 8,
 8,
 9,
 9,
 10,
 10,
 11,
 11,
 12,
 12,
 13,
 13,
 14,
 14,
 15,
 15,
 16,
 16,
 17,
 17,
 18,
 18,
 19,
 19,
 20,
 20,
 21,
 21,
 22,
 22,
 23,
 23,
 24,
 24,
 25,
 25,
 26,
 26,
 27,
 27,
 28,
 28,
 29,
 29,
 30,
 30,
 31,
 31,
 32,
 32,
 33,
 33,
 34,
 34,
 35,
 35,
 36,
 36,
 37,
 37,
 38,
 38,
 39,
 39,
 40,
 40,
 41,
 41,
 42,
 42,
 43,
 43,
 44,
 44,
 45,
 45,
 46,
 46,
 47,
 47,
 48,
 48,
 49,
 49,
 50,
 50,
 51,
 51,
 52,
 52,
 53,
 53,
 54,
 54,
 55,
 55,
 56,
 56,
 57,
 57,
 58,
 58,
 59,
 59,
 60,
 60,
 61,
 61,
 62,
 62,
 63,
 63,
 64,
 64,
 65,
 65,
 66,
 66,
 67,
 67,
 68,
 68,
 69,
 69,
 70,
 70,
 71,
 71,
 72,
 72,
 73,
 73,
 74,
 74,
 75,
 75,
 76,
 76,
 77,
 77,
 78,
 78,
 79,
 79,
 80,
 80,
 81,
 81,
 82,
 82,
 83,
 83,
 84,
 84,
 85,
 85,
 86,
 86,
 87,
 87,
 88,
 88,
 89,
 89,
 90,
 90,
 91,
 91,
 92,
 92,
 93,
 93,
 94,
 94,
 95,
 95,
 96,
 96,
 97,
 97,
 98,
 98,
 99,
 99,
 100,
 100,
 101,
 1

In [6]:
data_dict = {
        ('uctx', 'cu', 'user'): (udst, usrc),
        ('user', 'uc', 'uctx'): (usrc, udst),
        ('sctx', 'cs', 'serv'): (sdst, ssrc),
        ('serv', 'sc', 'sctx'): (ssrc, sdst),
        ('user', 'us', 'serv'): (msrc, mdst),
        ('serv', 'su', 'user'): (mdst, msrc)
}
data_dict.keys()

dict_keys([('uctx', 'cu', 'user'), ('user', 'uc', 'uctx'), ('sctx', 'cs', 'serv'), ('serv', 'sc', 'sctx'), ('user', 'us', 'serv'), ('serv', 'su', 'user')])

In [7]:
import dgl as d
graph = d.heterograph(data_dict=data_dict)
graph

Graph(num_nodes={'sctx': 3766, 'serv': 5825, 'uctx': 168, 'user': 339},
      num_edges={('sctx', 'cs', 'serv'): 17475, ('serv', 'sc', 'sctx'): 17475, ('serv', 'su', 'user'): 1827436, ('uctx', 'cu', 'user'): 678, ('user', 'uc', 'uctx'): 678, ('user', 'us', 'serv'): 1827436},
      metagraph=[('sctx', 'serv', 'cs'), ('serv', 'sctx', 'sc'), ('serv', 'user', 'su'), ('user', 'uctx', 'uc'), ('user', 'serv', 'us'), ('uctx', 'user', 'cu')])

In [8]:
graph.ndata.keys()

dict_keys([])

In [9]:
graph.edata.keys()

dict_keys([])

In [10]:
np.array([
    [1, 2],
    [3, 4],
    [2, 1],
    [4, 3]
])

array([[1, 2],
       [3, 4],
       [2, 1],
       [4, 3]])

In [11]:
import torch as t
RTValue = t.tensor(rts).float()
TPValue = t.tensor(tps).float()

graph.edata['RT'] = {('user', 'us', 'serv'): RTValue,
                     ('serv', 'su', 'user'): RTValue}

# graph.edata['TP'] = {('user', 'us', 'serv'): TPValue,
#                      ('serv', 'su', 'user'): TPValue}
graph

Graph(num_nodes={'sctx': 3766, 'serv': 5825, 'uctx': 168, 'user': 339},
      num_edges={('sctx', 'cs', 'serv'): 17475, ('serv', 'sc', 'sctx'): 17475, ('serv', 'su', 'user'): 1827436, ('uctx', 'cu', 'user'): 678, ('user', 'uc', 'uctx'): 678, ('user', 'us', 'serv'): 1827436},
      metagraph=[('sctx', 'serv', 'cs'), ('serv', 'sctx', 'sc'), ('serv', 'user', 'su'), ('user', 'uctx', 'uc'), ('user', 'serv', 'us'), ('uctx', 'user', 'cu')])

In [12]:
graph.ndata.keys(), graph.edata.keys()

(dict_keys([]), dict_keys(['RT']))

In [13]:
# graph.edata['RT'][('user', 'us', 'serv')].shape
graph.edata['RT'][('serv', 'su', 'user')].shape

torch.Size([1827436])

In [14]:
type(ufeats), len(ufeats), len(ufeats[0])  # ufeats 是 one hot 编码

(list, 339, 200)

In [15]:
graph.ndata['x'] = {
    'user': t.tensor(ufeats),
    'serv': t.tensor(sfeats)
}
graph

Graph(num_nodes={'sctx': 3766, 'serv': 5825, 'uctx': 168, 'user': 339},
      num_edges={('sctx', 'cs', 'serv'): 17475, ('serv', 'sc', 'sctx'): 17475, ('serv', 'su', 'user'): 1827436, ('uctx', 'cu', 'user'): 678, ('user', 'uc', 'uctx'): 678, ('user', 'us', 'serv'): 1827436},
      metagraph=[('sctx', 'serv', 'cs'), ('serv', 'sctx', 'sc'), ('serv', 'user', 'su'), ('user', 'uctx', 'uc'), ('user', 'serv', 'us'), ('uctx', 'user', 'cu')])

In [16]:
graph.ndata['x']['user'].shape, graph.ndata['x']['serv'].shape

(torch.Size([339, 200]), torch.Size([5825, 3800]))

In [17]:
graph.ndata.keys(), graph.edata.keys()

(dict_keys(['x']), dict_keys(['RT']))

In [18]:
graph.ntypes, graph.ndata.keys(), graph.edata.keys()

(['sctx', 'serv', 'uctx', 'user'], dict_keys(['x']), dict_keys(['RT']))

In [19]:
p = np.random.RandomState(seed=233).permutation(graph.number_of_edges('us'))

trainsize = int(density * graph.number_of_edges('us'))
trainIdx = p[: trainsize]
testIdx = p[trainsize:][: trainsize]

trainSubGraph = d.edge_subgraph(graph, edges={
    ('uctx', 'cu', 'user'): np.arange(0, graph.number_of_edges('cu')),
    ('user', 'uc', 'uctx'): np.arange(0, graph.number_of_edges('uc')),
    ('sctx', 'cs', 'serv'): np.arange(0, graph.number_of_edges('cs')),
    ('serv', 'sc', 'sctx'): np.arange(0, graph.number_of_edges('sc')),
    ('user', 'us', 'serv'): trainIdx,
    ('serv', 'su', 'user'): trainIdx},
)

testSubGraph = d.edge_subgraph(graph, edges={
    ('uctx', 'cu', 'user'): np.arange(0, graph.number_of_edges('cu')),
    ('user', 'uc', 'uctx'): np.arange(0, graph.number_of_edges('uc')),
    ('sctx', 'cs', 'serv'): np.arange(0, graph.number_of_edges('cs')),
    ('serv', 'sc', 'sctx'): np.arange(0, graph.number_of_edges('sc')),
    ('user', 'us', 'serv'): testIdx,
    ('serv', 'su', 'user'): testIdx},
)
trainGraph = trainSubGraph
testGraph = testSubGraph

In [20]:
trainGraph, testGraph

(Graph(num_nodes={'sctx': 3766, 'serv': 5825, 'uctx': 168, 'user': 339},
       num_edges={('sctx', 'cs', 'serv'): 17475, ('serv', 'sc', 'sctx'): 17475, ('serv', 'su', 'user'): 18274, ('uctx', 'cu', 'user'): 678, ('user', 'uc', 'uctx'): 678, ('user', 'us', 'serv'): 18274},
       metagraph=[('sctx', 'serv', 'cs'), ('serv', 'sctx', 'sc'), ('serv', 'user', 'su'), ('user', 'uctx', 'uc'), ('user', 'serv', 'us'), ('uctx', 'user', 'cu')]),
 Graph(num_nodes={'sctx': 3766, 'serv': 5825, 'uctx': 168, 'user': 339},
       num_edges={('sctx', 'cs', 'serv'): 17475, ('serv', 'sc', 'sctx'): 17475, ('serv', 'su', 'user'): 18274, ('uctx', 'cu', 'user'): 678, ('user', 'uc', 'uctx'): 678, ('user', 'us', 'serv'): 18274},
       metagraph=[('sctx', 'serv', 'cs'), ('serv', 'sctx', 'sc'), ('serv', 'user', 'su'), ('user', 'uctx', 'uc'), ('user', 'serv', 'us'), ('uctx', 'user', 'cu')]))

In [21]:
from datasets import QoSDataSetV3

trainset = QoSDataSetV3(trainGraph, True)

/Users/zengyuxiang/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:232: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)


# Dataset内部探索

In [22]:
graph = trainGraph

In [23]:
bipart = d.node_type_subgraph(graph, ['user', 'serv'])
bipart

Graph(num_nodes={'serv': 5825, 'user': 339},
      num_edges={('serv', 'su', 'user'): 18274, ('user', 'us', 'serv'): 18274},
      metagraph=[('serv', 'user', 'su'), ('user', 'serv', 'us')])

In [24]:
links = graph.all_edges(etype='us')
links

(tensor([  0,   1,   2,  ..., 324, 129,  98]),
 tensor([3105, 1843, 1054,  ..., 3143,  559, 1477]))

In [25]:
labels = graph.edata['RT'][('user', 'us', 'serv')]
labels

tensor([0.4020, 7.7170, 0.3460,  ..., 0.3370, 0.1250, 0.7230])

In [26]:
# 数据集获取全过程
# centerUser, centerServ = int(self.links[0][idx]), int(self.links[1][idx])
# label = self.labels[idx]
# subgraph = self.extractSubgraph(graph=self.graph, u=centerUser, s=centerServ)
# ngraph = self.extractNeighSubgraph(graph=self.graph, u=centerUser, s=centerServ)

# 获取子图过程

In [27]:
idx = 0

In [28]:
# 首先获得中心节点
# graph=self.graph, u=centerUser, s=centerServ
_fringeUsersCache = {}  # user邻居？
_fringeServsCache = {}  # serv邻居？
centerUser, centerServ = int(links[0][idx]), int(links[1][idx])
u_fringe = centerUser
v_fringe = centerServ
u_fringe, v_fringe

(0, 3105)

In [29]:
user, serv = u_fringe, v_fringe
if serv not in _fringeUsersCache:
    neighUser = bipart.in_edges(serv, etype='us')[0]
    _fringeUsersCache[serv] = neighUser
neighUser = _fringeUsersCache[serv]
neighUser = np.setdiff1d(neighUser, user)  # 从邻接用户节点的入边索引中移除已经存在的用户节点 user。

In [30]:
if user not in _fringeServsCache:
    neighServ = graph.in_edges(user, etype='su')[0]
    _fringeServsCache[user] = neighServ
neighServ = _fringeServsCache[user]
neighServ = np.setdiff1d(neighServ, serv)

In [31]:
_fringeUsersCache, _fringeServsCache

({3105: tensor([  0, 135,  10, 328,  95,   3, 220, 237])},
 {0: tensor([3105, 1274, 1753, 5157, 4215, 2536, 4607, 1794, 2418, 4542, 2511, 4036,
          3943, 5036,  783, 5419, 3426, 3906, 4948,  973, 4900,  124, 4422, 1448,
          2928,  306, 3205, 1815, 4009, 5261,  236, 3179, 3940, 4569, 3603,  359,
          1392, 2120, 1812, 1914])})

In [32]:
neighUser, neighServ

(array([  3,  10,  95, 135, 220, 237, 328]),
 array([ 124,  236,  306,  359,  783,  973, 1274, 1392, 1448, 1753, 1794,
        1812, 1815, 1914, 2120, 2418, 2511, 2536, 2928, 3179, 3205, 3426,
        3603, 3906, 3940, 3943, 4009, 4036, 4215, 4422, 4542, 4569, 4607,
        4900, 4948, 5036, 5157, 5261, 5419]))

In [33]:
t.tensor([u_fringe]), t.from_numpy(neighUser)

(tensor([0]), tensor([  3,  10,  95, 135, 220, 237, 328]))

In [34]:
# 构建子图的节点集合
u_nodes = t.cat((t.tensor([u_fringe]), t.from_numpy(neighUser)))
v_nodes = t.cat((t.tensor([v_fringe]), t.from_numpy(neighServ)))
u_nodes, v_nodes

(tensor([  0,   3,  10,  95, 135, 220, 237, 328]),
 tensor([3105,  124,  236,  306,  359,  783,  973, 1274, 1392, 1448, 1753, 1794,
         1812, 1815, 1914, 2120, 2418, 2511, 2536, 2928, 3179, 3205, 3426, 3603,
         3906, 3940, 3943, 4009, 4036, 4215, 4422, 4542, 4569, 4607, 4900, 4948,
         5036, 5157, 5261, 5419]))

In [35]:
# 从原始图中提取子图
sg = d.node_subgraph(bipart, nodes={
        'user': u_nodes,
        'serv': v_nodes,
})  # 子图的类型为d.DGLGraph
sg

Graph(num_nodes={'serv': 40, 'user': 8},
      num_edges={('serv', 'su', 'user'): 48, ('user', 'us', 'serv'): 48},
      metagraph=[('serv', 'user', 'su'), ('user', 'serv', 'us')])

In [36]:
# 获取子图中用户节点的子图索引
su = sg.nodes('user')[sg.ndata[d.NID]['user'] == u_fringe]
 # 获取子图中服务节点的子图索引
sv = sg.nodes('serv')[sg.ndata[d.NID]['serv'] == v_fringe]
su, sv

(tensor([0]), tensor([0]))

In [37]:
# 为子图节点添加'label'属性，初始值为全1
sg.ndata['label'] = {
    'user': t.ones(sg.number_of_nodes('user')),
    'serv': t.ones(sg.number_of_nodes('serv'))
}
sg

Graph(num_nodes={'serv': 40, 'user': 8},
      num_edges={('serv', 'su', 'user'): 48, ('user', 'us', 'serv'): 48},
      metagraph=[('serv', 'user', 'su'), ('user', 'serv', 'us')])

In [38]:
# 将用户fringe的'label'值设为0
# 将服务fringe的'label'值设为0
sg.ndata['label']['user'][su] = 0
sg.ndata['label']['serv'][sv] = 0

In [39]:
# 根据用户和服务的子图索引，创建边的掩码
mask_us_edge = sg.edge_ids(su, sv, etype='us')
mask_su_edge = sg.edge_ids(sv, su, etype='su')

In [40]:
# 为边添加'mask'属性，初始值为全1
sg.edges['us'].data['mask'] = t.ones(sg.number_of_edges(etype='us'))
sg.edges['su'].data['mask'] = t.ones(sg.number_of_edges(etype='su'))

In [41]:
# 将用户和服务之间的边的'mask'值设为0
sg.edges['us'].data['mask'][mask_us_edge] = 0
sg.edges['su'].data['mask'][mask_su_edge] = 0


In [42]:
u_fringe = centerUser
v_fringe = centerServ
u_fringe, v_fringe

(0, 3105)

In [43]:
_neighUsersCache = {}
_neighServsCache = {}

In [44]:
# 如果用户不在邻居用户缓存中
if u_fringe not in _neighUsersCache:
    # 获取用户的上下文节点（出边类型为'uc'）
    uctxNodes = graph.out_edges(u_fringe, etype='uc')[1]

    # 获取上下文节点的相邻用户（入边类型为'cu'）
    neighUser = graph.out_edges(uctxNodes, etype='cu')[1]

    # 从相邻用户中排除当前用户
    neighUser = np.setdiff1d(neighUser, u_fringe)

    # 将结果缓存到邻居用户缓存中
    _neighUsersCache[u_fringe] = [uctxNodes, neighUser]

# 从邻居用户缓存中获取上下文节点和相邻用户
uctxNodes, neighUser = _neighUsersCache[u_fringe]

# 设置邻居用户的采样大小
neighSize = 35  # 或者设置为60

# 如果相邻用户数量超过了采样大小
if len(neighUser) > neighSize:
    # 获取相邻用户的出边度数，并将度数归一化
    degrees = graph.out_degrees(neighUser, etype='us').numpy()
    sumd = np.sum(degrees)
    degrees = degrees / sumd

    # 计算非零度数的数量
    non_zero = len(degrees[degrees > 0])

    # 设置采样大小为邻居大小和非零度数的较小值
    sample_size = min(neighSize, non_zero)

    # 从相邻用户中随机采样一部分节点
    neighUser = np.random.choice(neighUser, sample_size, replace=False, p=degrees)

In [45]:
if v_fringe not in _neighServsCache:
    sctxNodes = graph.out_edges(v_fringe, etype='sc')[1]
    neighServ = graph.out_edges(sctxNodes, etype='cs')[1]
    neighServ = np.setdiff1d(neighServ, v_fringe)
    _neighServsCache[v_fringe] = [sctxNodes, neighServ]
sctxNodes, neighServ = _neighServsCache[v_fringe]
neighSize = 85  # 300
if len(neighServ) > neighSize:
    degrees = graph.out_degrees(neighServ, etype='su').numpy()
    sumd = np.sum(degrees)
    degrees = degrees / sumd
    non_zero = len(degrees[degrees > 0])
    sample_size = min(neighSize, non_zero)
    neighServ = np.random.choice(
        neighServ, sample_size, replace=False, p=degrees)

In [46]:
neighUsers = neighUser
neighServs = neighServ

In [47]:
neighUsers, neighServs

(array([296]),
 array([3036, 3129, 3114, 2694, 2783, 2700, 2989, 5600, 2820, 2777, 2834,
        3005, 2936, 2802, 2710, 2693, 3127, 1405, 3043, 2947, 2943, 2716,
        2995, 1410, 3054, 2755, 2875, 3115, 4067, 3093, 2950, 2728, 2922,
        3097, 2690, 2769, 1269, 2927, 2957, 2773, 3081, 5597, 2831, 1409,
        3092, 2846, 3116, 5254, 2848, 2697, 3104, 3004, 2939, 3047, 2824,
        3142, 2873, 2708, 2916, 1089, 3141, 2951, 3028, 2797, 2847, 2754,
        3011, 2914, 3074, 2825, 2992, 2775, 2770, 3021, 3040, 2713, 2774,
        2785, 3110, 1412, 2771, 2839, 2733, 2968, 3061]))

In [48]:
ngraph = d.node_subgraph(bipart, nodes={
    'user': neighUsers,
    'serv': neighServs
})  # type:d.DGLGraph
ngraph

Graph(num_nodes={'serv': 85, 'user': 1},
      num_edges={('serv', 'su', 'user'): 1, ('user', 'us', 'serv'): 1},
      metagraph=[('serv', 'user', 'su'), ('user', 'serv', 'us')])

In [49]:
ngraph.nodes['user'].data

{'x': tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.]]), '_ID': tensor([296])}

In [50]:
ufeats = t.nn.Linear(200, 64, bias = False)
sfeats = t.nn.Linear(3800, 64, bias = False)
usermsg = t.nn.Linear(64 + 1, 64, bias=True)
servmsg = t.nn.Linear(64 + 1, 64, bias=True)

In [51]:
ngraph.nodes['user'].data['in'] = ufeats(ngraph.nodes['user'].data['x'])
ngraph.nodes['serv'].data['in'] = sfeats(ngraph.nodes['serv'].data['x'])
ngraph

Graph(num_nodes={'serv': 85, 'user': 1},
      num_edges={('serv', 'su', 'user'): 1, ('user', 'us', 'serv'): 1},
      metagraph=[('serv', 'user', 'su'), ('user', 'serv', 'us')])

In [52]:
ngraph.nodes['user'].data['in'].shape

torch.Size([1, 64])

In [53]:
ngraph.edges['us'], ngraph.edges['su']

(EdgeSpace(data={'RT': tensor([0.7000]), '_ID': tensor([15717])}),
 EdgeSpace(data={'RT': tensor([0.7000]), '_ID': tensor([15717])}))

In [54]:
def userToServ(edges):
    # 提取源节点（用户）的特征和边的属性
    userEmbs = edges.src['in']
    attrQoS = edges.data['RT'].reshape((-1, 1))
    # 将用户特征和边属性连接
    input = t.cat((userEmbs, attrQoS), -1)
    # 计算用户消息
    eventMsg = usermsg(input)
    return {'eventMsg': eventMsg}
def servToUser(edges):
    # 提取源节点（服务）的特征和边的属性
    servEmbs = edges.src['in']
    attrQoS = edges.data['RT'].reshape((-1, 1))
    # 将服务特征和边属性连接
    input = t.cat((servEmbs, attrQoS), -1)
    # 计算服务消息
    eventMsg = servmsg(input)
    return {'eventMsg': eventMsg}

In [55]:
import dgl.function as fn
ngraph.update_all(userToServ, fn.mean('eventMsg', 'out'), etype='us')
ngraph.update_all(servToUser, fn.mean('eventMsg', 'out'), etype='su')

In [56]:
ngraph.edata

defaultdict(<class 'dict'>, {'RT': {('serv', 'su', 'user'): tensor([0.7000]), ('user', 'us', 'serv'): tensor([0.7000])}, '_ID': {('serv', 'su', 'user'): tensor([15717]), ('user', 'us', 'serv'): tensor([15717])}})

In [57]:
user_read_out = d.readout_nodes(ngraph, 'out', ntype = 'user', op = 'mean')
user_read_out, user_read_out.shape

(tensor([[-0.0448,  0.0818, -0.1059,  0.0483, -0.0321,  0.1828, -0.0733,  0.0279,
           0.0243, -0.0476, -0.0520,  0.0495,  0.0858, -0.0320, -0.0951, -0.0049,
          -0.1588, -0.0621,  0.0542,  0.1143,  0.1258,  0.1442,  0.0360, -0.0095,
          -0.1123, -0.0961,  0.0035, -0.0865,  0.1234,  0.0400,  0.0220, -0.0367,
           0.0459, -0.0538,  0.0623, -0.0171, -0.1022,  0.0766,  0.0429, -0.0059,
          -0.1453,  0.1296,  0.0614, -0.0144,  0.0076,  0.0210,  0.0411, -0.0329,
           0.0393, -0.0726, -0.1484,  0.0178, -0.0264, -0.0008, -0.0035,  0.0479,
          -0.0947,  0.0579, -0.0064,  0.0513,  0.0480, -0.0874, -0.0459,  0.0234]],
        grad_fn=<DivBackward0>),
 torch.Size([1, 64]))

In [58]:
act = t.nn.LeakyReLU()
user_readout = act(d.readout_nodes(ngraph, 'out', ntype='user', op='mean'))
serv_readout = act(d.readout_nodes(ngraph, 'out', ntype='serv', op='mean'))
serv_readout.shape

torch.Size([1, 64])

In [59]:
user_serv_readout = t.cat((user_readout, serv_readout), -1)
user_serv_readout.shape

torch.Size([1, 128])

In [60]:

"""
def forward(self, batchGraph, neighGraph=None):
    ################################################################
    # Subgraph

    # 初始化邻居的readout表示
    neigh_readout = None

    # 将邻居用户节点特征进行线性变换，得到输入特征
    neighGraph.nodes['user'].data['in'] = self.ufeats(neighGraph.nodes['user'].data['x'])

    # 将邻居服务节点特征进行线性变换，得到输入特征
    neighGraph.nodes['serv'].data['in'] = self.sfeats(neighGraph.nodes['serv'].data['x'])

    # 使用SubGraphConv模块进行邻居图的计算，得到邻居的readout表示
    neigh_readout = self.neighbourEncoder(neighGraph)

    ################################################################
    # Neighbor

    # 将用户节点特征进行线性变换，得到输入特征
    batchGraph.nodes['user'].data['in'] = self.ufeats(batchGraph.nodes['user'].data['x'])

    # 将服务节点特征进行线性变换，得到输入特征
    batchGraph.nodes['serv'].data['in'] = self.sfeats(batchGraph.nodes['serv'].data['x'])

    # 使用Interaction模块进行用户和服务的交互，得到用户和服务的Embeddings
    UserEmbeds, ServEmbeds = self.interactionEncoder(batchGraph)

    # 根据标签筛选用户和服务Embeddings
    users = batchGraph.ndata['label']['user'] == 0
    items = batchGraph.ndata['label']['serv'] == 0

    # 拼接用户和服务Embeddings
    x = t.cat([UserEmbeds[users], ServEmbeds[items]], -1)

    # 使用LinkPrediction模块进行链接预测
    rtLink = self.Link(x, neigh_readout)

    # 返回链接预测结果
    return rtLink

"""

"\ndef forward(self, batchGraph, neighGraph=None):\n    ################################################################\n    # Subgraph\n\n    # 初始化邻居的readout表示\n    neigh_readout = None\n\n    # 将邻居用户节点特征进行线性变换，得到输入特征\n    neighGraph.nodes['user'].data['in'] = self.ufeats(neighGraph.nodes['user'].data['x'])\n\n    # 将邻居服务节点特征进行线性变换，得到输入特征\n    neighGraph.nodes['serv'].data['in'] = self.sfeats(neighGraph.nodes['serv'].data['x'])\n\n    # 使用SubGraphConv模块进行邻居图的计算，得到邻居的readout表示\n    neigh_readout = self.neighbourEncoder(neighGraph)\n\n    ################################################################\n    # Neighbor\n\n    # 将用户节点特征进行线性变换，得到输入特征\n    batchGraph.nodes['user'].data['in'] = self.ufeats(batchGraph.nodes['user'].data['x'])\n\n    # 将服务节点特征进行线性变换，得到输入特征\n    batchGraph.nodes['serv'].data['in'] = self.sfeats(batchGraph.nodes['serv'].data['x'])\n\n    # 使用Interaction模块进行用户和服务的交互，得到用户和服务的Embeddings\n    UserEmbeds, ServEmbeds = self.interactionEncoder(batchGraph)\n\n    # 根据标

In [61]:
"""
class SubGraphConv(t.nn.Module):

    def __init__(self, dim, order=1, type='RT'):
        super(SubGraphConv, self).__init__()

        # 初始化SubGraphConv模块
        self.order = order
        self.dim = dim
        self.type = type

        # 用户消息传递线性层
        self.usermsg = t.nn.Linear(self.dim + 1, self.dim, bias=True)

        # 服务消息传递线性层
        self.servmsg = t.nn.Linear(self.dim + 1, self.dim, bias=True)

        # LeakyReLU激活函数
        self.act = t.nn.LeakyReLU()

    def userToServ(self, edges):
        # 获取源节点（用户）的输入特征和边的属性
        userEmbs = edges.src['in']
        attrQoS = edges.data[self.type].reshape((-1, 1))

        # 将用户Embedding和边的属性拼接
        input = t.cat((userEmbs, attrQoS), -1)

        # 使用用户消息传递线性层
        eventMsg = self.usermsg(input)

        return {'eventMsg': eventMsg}

    def servToUser(self, edges):
        # 获取源节点（服务）的输入特征和边的属性
        servEmbs = edges.src['in']
        attrQoS = edges.data[self.type].reshape((-1, 1))

        # 将服务Embedding和边的属性拼接
        input = t.cat((servEmbs, attrQoS), -1)

        # 使用服务消息传递线性层
        eventMsg = self.servmsg(input)

        return {'eventMsg': eventMsg}

    def forward(self, bGraph):
        # 将输入的Batch图赋值给本地图对象
        graph = bGraph  # type:d.DGLGraph

        # 用户到服务的消息传递
        graph.update_all(self.userToServ, fn.mean('eventMsg', 'out'), etype='us')

        # 服务到用户的消息传递
        graph.update_all(self.servToUser, fn.mean('eventMsg', 'out'), etype='su')

        # 读出用户节点和服务节点的消息，求均值并使用LeakyReLU激活函数
        user_readout = self.act(d.readout_nodes(graph, 'out', ntype='user', op='mean'))
        serv_readout = self.act(d.readout_nodes(graph, 'out', ntype='serv', op='mean'))

        # 拼接用户和服务的读出结果
        user_serv_readout = t.cat((user_readout, serv_readout), -1)

        # 返回用户和服务的读出结果
        return user_serv_readout

"""


"\nclass SubGraphConv(t.nn.Module):\n\n    def __init__(self, dim, order=1, type='RT'):\n        super(SubGraphConv, self).__init__()\n\n        # 初始化SubGraphConv模块\n        self.order = order\n        self.dim = dim\n        self.type = type\n\n        # 用户消息传递线性层\n        self.usermsg = t.nn.Linear(self.dim + 1, self.dim, bias=True)\n\n        # 服务消息传递线性层\n        self.servmsg = t.nn.Linear(self.dim + 1, self.dim, bias=True)\n\n        # LeakyReLU激活函数\n        self.act = t.nn.LeakyReLU()\n\n    def userToServ(self, edges):\n        # 获取源节点（用户）的输入特征和边的属性\n        userEmbs = edges.src['in']\n        attrQoS = edges.data[self.type].reshape((-1, 1))\n\n        # 将用户Embedding和边的属性拼接\n        input = t.cat((userEmbs, attrQoS), -1)\n\n        # 使用用户消息传递线性层\n        eventMsg = self.usermsg(input)\n\n        return {'eventMsg': eventMsg}\n\n    def servToUser(self, edges):\n        # 获取源节点（服务）的输入特征和边的属性\n        servEmbs = edges.src['in']\n        attrQoS = edges.data[self.type].reshape((-1, 